### Multimodal RAG (PDF With Images)

![image.png](image.png)

In [1]:
import fitz  # PyMuPDF
from langchain_core.documents import Document
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
import numpy as np
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.schema.messages import HumanMessage
from sklearn.metrics.pairwise import cosine_similarity
import os
import base64
import io
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

f:\Datascience_projects\Multi_Model_RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
### Clip model
import os
from dotenv import load_dotenv
load_dotenv()

##set up the environment variables
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

## initialize the Clip model for unified embedding
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model.eval()



### clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

This loads the pre-trained CLIP model (Vision Transformer base, patch size 32) from Hugging Face. The model can generate embeddings for both images and text, enabling comparison between them in a shared embedding space.

### clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

This loads the corresponding processor, which handles preprocessing (tokenization for text, resizing/normalization for images) so that inputs are compatible with the CLIP model.
Together, these are used for unified multimodal (text and image) embedding and retrieval tasks.

https://huggingface.co/docs/transformers/model_doc/clip?usage=Pipeline#transformers.CLIPProcessor

In [ ]:
### Embedding function
def embed_image(image_data):
    """Embed Image Using Clip"""
    if isinstance(image_data,str): #if path
        image = Image.open(image_data).convert("RGB")
    else: #If PIL Image
        image = image_data
    
    clip_processor(images=image,return_tensors='pt')  # pt means pytorch tensors
    #It takes the image and applies the necessary preprocessing steps 
    # (such as resizing, normalization, and conversion to tensor format) required by the CLIP model.
    with torch.no_grad():
        features = clip_model.get_image_features(**inputs)
        ## Normalize embeddings to unit vector
        features = features/ features.norm(dim=-1,keepdim=True)
        return features.squeeze().numpy()

#with torch.no_grad():
# Disables gradient calculation, which is not needed for inference and saves memory/computation.

# features = clip_model.get_image_features(**inputs)
# Passes the processed image tensor to the CLIP model to get its embedding vector.

# features = features / features.norm(dim=-1, keepdim=True)
# Normalizes the embedding vector to have unit length (L2 norm = 1), which is useful for similarity comparisons.

# return features.squeeze().numpy()
# Removes any singleton dimensions from the tensor and converts it to a NumPy array for further processing or storage.
# This sequence extracts a normalized image embedding from the CLIP model for use in retrieval or similarity tasks.
    
def embed_text(text):
    """Embed text using CLIP."""
    inputs = clip_processor(
        text=text, 
        return_tensors="pt", 
        padding=True,
        truncation=True,
        max_length=77  # CLIP's max token length
    )
    with torch.no_grad():
        features = clip_model.get_text_features(**inputs)
        # Normalize embeddings
        features = features / features.norm(dim=-1, keepdim=True)
        return features.squeeze().numpy()

The package fitz is the Python interface for PyMuPDF, a library used to read, manipulate, and extract information from PDF documents and other supported file formats

In [4]:
## Process PDF
pdf_path="multimodal-sample.pdf"
doc=fitz.open(pdf_path)
# Storage for all documents and embeddings
all_docs = []
all_embeddings = []
image_data_store = {}  # Store actual image data for LLM

# Text splitter
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

In [5]:
doc

Document('multimodal-sample.pdf')

In [ ]:
for i,page in enumerate(doc):
    ## process text
    text=page.get_text()
    if text.strip():
        ##create temporary document for splitting
        temp_doc = Document(page_content=text, metadata={"page": i, "type": "text"})
        text_chunks = splitter.split_documents([temp_doc])

        #Embed each chunk using CLIP
        for chunk in text_chunks:
            embedding = embed_text(chunk.page_content)
            all_embeddings.append(embedding)
            all_docs.append(chunk)



    ## process images
    ##Three Important Actions:

    ##Convert PDF image to PIL format
    ##Store as base64 for GPT-4V (which needs base64 images)
    ##Create CLIP embedding for retrieval

    for img_index, img in enumerate(page.get_images(full=True)):
        try:
            xref = img[0]#Gets the reference number (xref) for the image object in the PDF.
            base_image = doc.extract_image(xref)#Extracts the image data from the PDF using the xref.
            image_bytes = base_image["image"]#Retrieves the raw image bytes from the extracted image data.
#Raw image bytes refer to the binary data that directly represents the image file, 
# before it is decoded or processed. This data includes all the pixel information and file encoding (such as PNG or JPEG format) as stored in the PDF

            # Convert to PIL Image so you can process, display, or transform the image using Python code.
            pil_image = Image.open(io.BytesIO(image_bytes)).convert("RGB")
            
            # Create unique identifier
            image_id = f"page_{i}_img_{img_index}"
            
            # Store image as base64 for later use with GPT-4V
            buffered = io.BytesIO()#Creates an in-memory buffer to store the image
            pil_image.save(buffered, format="PNG")#Saves the PIL image to the buffer in PNG format.
            img_base64 = base64.b64encode(buffered.getvalue()).decode()#Encodes the buffered image as a base64 string for use with models like GPT-4V.
#buffered.getvalue() gets the raw bytes of the image from the buffer.
# base64.b64encode(...) encodes those bytes into a base64 format, which is a way to represent binary data as text.
# .decode() converts the base64 bytes into a regular string.
            image_data_store[image_id] = img_base64#Stores the base64-encoded image string in a dictionary with a unique identifier.
            
            # Embed image using CLIP
            embedding = embed_image(pil_image)
            all_embeddings.append(embedding)
            
            # Create document for image
            image_doc = Document(
                page_content=f"[Image: {image_id}]",
                metadata={"page": i, "type": "image", "image_id": image_id}
            )
            all_docs.append(image_doc)
            
        except Exception as e:
            print(f"Error processing image {img_index} on page {i}: {e}")
            continue

doc.close()


In [ ]:
all_embeddings

In [ ]:
all_docs

In [ ]:
# Create unified FAISS vector store with CLIP embeddings
embeddings_array = np.array(all_embeddings)
embeddings_array

In [ ]:
(all_docs, all_embeddings)  

In [ ]:
vector_store = FAISS.from_embeddings(
    text_embeddings=[(doc.page_content,emb) for doc, emb in zip(all_docs, all_embeddings)],
    embedding=None, #We're using pre computed embeddings
    metadatas=[doc.metadata for doc in all_docs]
    )

In [ ]:
# Initialize GPT-4 Vision model
llm = init_chat_model("openai:gpt-4.1")
llm

In [ ]:
def retrieve_multimodal(query, k=5):
    """Unified retrieval using CLIP embeddings for both text and images."""
    # Embed query using CLIP
    query_embedding = embed_text(query)
    
    # Search in unified vector store
    results = vector_store.similarity_search_by_vector(
        embedding=query_embedding,
        k=k
    )
    
    return results

In [ ]:
def create_multimodal_message(query, retrieved_docs):
    """Create a message with both text and images for GPT-4V."""
    content = []
    
    # Add the query
    content.append({
        "type": "text",
        "text": f"Question: {query}\n\nContext:\n"
    })
    
    # Separate text and image documents
    text_docs = [doc for doc in retrieved_docs if doc.metadata.get("type") == "text"]
    image_docs = [doc for doc in retrieved_docs if doc.metadata.get("type") == "image"]
    
    # Add text context
    if text_docs:
        text_context = "\n\n".join([
            f"[Page {doc.metadata['page']}]: {doc.page_content}"
            for doc in text_docs
        ])
        content.append({
            "type": "text",
            "text": f"Text excerpts:\n{text_context}\n"
        })
    
    # Add images
    for doc in image_docs:
        image_id = doc.metadata.get("image_id")
        if image_id and image_id in image_data_store:
            content.append({
                "type": "text",
                "text": f"\n[Image from page {doc.metadata['page']}]:\n"
            })
            content.append({
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/png;base64,{image_data_store[image_id]}"
                }
            })
    
    # Add instruction
    content.append({
        "type": "text",
        "text": "\n\nPlease answer the question based on the provided text and images."
    })
    
    return HumanMessage(content=content)

In [ ]:
def multimodal_pdf_rag_pipeline(query):
    """Main pipeline for multimodal RAG."""
    # Retrieve relevant documents
    context_docs = retrieve_multimodal(query, k=5)
    
    # Create multimodal message
    message = create_multimodal_message(query, context_docs)
    
    # Get response from GPT-4V
    response = llm.invoke([message])
    
    # Print retrieved context info
    print(f"\nRetrieved {len(context_docs)} documents:")
    for doc in context_docs:
        doc_type = doc.metadata.get("type", "unknown")
        page = doc.metadata.get("page", "?")
        if doc_type == "text":
            preview = doc.page_content[:100] + "..." if len(doc.page_content) > 100 else doc.page_content
            print(f"  - Text from page {page}: {preview}")
        else:
            print(f"  - Image from page {page}")
    print("\n")
    
    return response.content

In [ ]:
if __name__ == "__main__":
    # Example queries
    queries = [
        "What does the chart on page 1 show about revenue trends?",
        "Summarize the main findings from the document",
        "What visual elements are present in the document?"
    ]
    
    for query in queries:
        print(f"\nQuery: {query}")
        print("-" * 50)
        answer = multimodal_pdf_rag_pipeline(query)
        print(f"Answer: {answer}")
        print("=" * 70)